imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import requests

helper function to format response objects into pandas dataframes

In [2]:
def call_endpoint(url, max_level=3, include_new_player_attributes=False):
    '''
    takes: 
        - url (str): the API endpoint to call
        - max_level (int): level of json normalizing to apply
        - include_player_attributes (bool): whether to include player object attributes in the returned dataframe
    returns:
        - df (pd.DataFrame): a dataframe of the call response content
    '''
    resp = requests.get(url).json()
    data = pd.json_normalize(resp['data'], max_level=max_level)
    included = pd.json_normalize(resp['included'], max_level=max_level)
    if include_new_player_attributes:
        inc_cop = included[included['type'] == 'new_player'].copy().dropna(axis=1)
        data = pd.merge(data, inc_cop, how='left', left_on=['relationships.new_player.data.id','relationships.new_player.data.type'], right_on=['id','type'], suffixes=('', '_new_player'))
    return data

calling leagues endpoint

In [16]:
url = 'http://partner-api.prizepicks.com/leagues' # leagues that are live on our board
df = call_endpoint(url)
df.head(3)

,id,type,attributes.name,attributes.rank,attributes.active,attributes.created_at,attributes.updated_at,attributes.icon,attributes.image_url,attributes.projections_count,relationships.projection_filters.data
0,25,league,NFL2H,0,True,2019-02-03T20:00:47.421-05:00,2021-09-09T23:05:28.930-04:00,football,https://static.prizepicks.com/images/leagues/f...,0,[]
1,9,league,NFL,1,True,2018-07-19T13:26:11.622-04:00,2021-09-10T10:11:40.728-04:00,football,https://static.prizepicks.com/images/leagues/f...,289,"[{'id': '1', 'type': 'projection_filter'}, {'i..."
2,150,league,CFB2H,2,True,2020-09-23T10:34:52.470-04:00,2021-09-10T10:11:40.732-04:00,football,https://static.prizepicks.com/images/leagues/f...,0,[]


calling projections endpoint

In [14]:
url = 'https://partner-api.prizepicks.com/projections?per_page=1000' # use per_page=1000 to avoid pagination issues
df = call_endpoint(url)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type
0,252051,projection,57.0,None,2021-09-09T16:13:27.533-04:00,2021-09-09T20:37:32.162-04:00,Round 2 (Par 63),2021-09-10T14:12:00.000-04:00,pre_game,2021-09-09T12:00:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,157,league,None,None,14,stat_type,28506,new_player
1,252414,projection,36.0,None,2021-09-09T23:16:19.243-04:00,2021-09-09T23:46:36.843-04:00,Auto Parts 400,2021-09-11T19:30:00.000-04:00,pre_game,2020-08-08T22:30:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,4,league,None,None,14,stat_type,22,new_player
2,252383,projection,37.5,None,2021-09-09T22:51:20.489-04:00,2021-09-10T05:21:57.683-04:00,Go Bowling 250,2021-09-11T14:30:00.000-04:00,pre_game,2021-06-18T15:30:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,137,league,None,None,14,stat_type,16081,new_player


calling projections endpoint with filters

In [ ]:
url = 'https://partner-api.prizepicks.com/projections?league_id=9&per_page=10&single_stat=true'
df = call_endpoint(url)
df.head(3)

calling projections endpoint with filters and including new_player level data (like name and team)

In [17]:
url = 'https://partner-api.prizepicks.com/projections?league_id=9&per_page=1000&single_stat=true'
df = call_endpoint(url, include_new_player_attributes=True)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type,id_new_player,type_new_player,attributes.name,attributes.created_at_new_player,attributes.updated_at_new_player,attributes.image_url,attributes.position,attributes.team,attributes.fantasy_data_id,attributes.league_id,attributes.league,relationships.league.data.id_new_player,relationships.league.data.type_new_player
0,238931,projection,0.5,None,2021-08-23T17:25:49.983-04:00,2021-09-10T01:44:21.721-04:00,CLE,2021-09-12T16:25:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,True,0.95,2021-09-12T01:00:00.000-04:00,None,None,Pass Yards,Single Stat,9,league,None,None,16,stat_type,1093,new_player,1093,new_player,Patrick Mahomes,2018-09-20T14:27:59.218-04:00,2020-07-30T18:28:24.765-04:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,KC,18890.0,9.0,NFL,9,league
1,251417,projection,47.5,None,2021-09-08T13:46:41.807-04:00,2021-09-10T09:57:14.695-04:00,ATL,2021-09-12T13:00:00.000-04:00,pre_game,2021-04-01T19:05:00.000-04:00,21,False,NaN,None,None,None,Rush Yards,Single Stat,9,league,None,None,15,stat_type,24941,new_player,24941,new_player,Jalen Hurts,2020-12-08T07:22:24.176-05:00,2020-12-08T07:22:24.176-05:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,PHI,21831.0,9.0,NFL,9,league
2,250238,projection,22.5,None,2021-09-07T18:25:43.089-04:00,2021-09-07T19:28:27.007-04:00,ATL,2021-09-12T13:00:00.000-04:00,pre_game,2021-08-26T00:00:00.000-04:00,21,False,NaN,None,None,None,Fantasy Score,Fantasy Score,9,league,None,None,14,stat_type,24941,new_player,24941,new_player,Jalen Hurts,2020-12-08T07:22:24.176-05:00,2020-12-08T07:22:24.176-05:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,PHI,21831.0,9.0,NFL,9,league


In [20]:
df[df['attributes.name'] == 'Jameis Winston']

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type,id_new_player,type_new_player,attributes.name,attributes.created_at_new_player,attributes.updated_at_new_player,attributes.image_url,attributes.position,attributes.team,attributes.fantasy_data_id,attributes.league_id,attributes.league,relationships.league.data.id_new_player,relationships.league.data.type_new_player
257,251514,projection,259.5,None,2021-09-08T18:20:49.640-04:00,2021-09-08T18:51:40.808-04:00,GB,2021-09-12T16:25:00.000-04:00,pre_game,2021-09-08T14:10:00.000-04:00,132,False,NaN,None,None,None,Pass Yards,Single Stat,9,league,None,None,16,stat_type,6569,new_player,6569,new_player,Jameis Winston,2019-09-03T12:12:14.579-04:00,2020-09-01T18:13:45.759-04:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,NO,16762.0,9.0,NFL,9,league
258,250415,projection,17.0,None,2021-09-07T22:39:10.701-04:00,2021-09-08T12:14:09.845-04:00,GB,2021-09-12T16:25:00.000-04:00,pre_game,2021-08-26T00:00:00.000-04:00,132,False,NaN,None,None,None,Fantasy Score,Fantasy Score,9,league,None,None,14,stat_type,6569,new_player,6569,new_player,Jameis Winston,2019-09-03T12:12:14.579-04:00,2020-09-01T18:13:45.759-04:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,NO,16762.0,9.0,NFL,9,league
